### 0) Importando pacotes:

In [1]:
import warnings
import os
import copy
import glob
import random
from termcolor import colored
import numpy as np
import pandas as pd
import xarray as xr 
warnings.filterwarnings("ignore")

/home/andre/mlpy3/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


### 1) Importando o código com a função que puxa os dados dos caminhos e pastas corretos:

In [ ]:
import EscolhaInicial5
outputs = EscolhaInicial5.instrumento()

path = outputs['Caminho_instrumento']
path_nc = outputs['Lista_netcdfs']
instrumento = outputs['Instrumento']
pasta_dfs = outputs['Caminho_para_dataframes']
var1 = outputs['Variavel']
var1_unity = outputs['Unidade']
com_cortes = ['aosnephdry1mM1','aosnephdry1mS1','aospsap3w1mM1','aospsap3w1mS1']
if instrumento in com_cortes:
    corte = outputs['Corte']
print('          ')
print('Lendo dados do instrumento {0}. A variavel é {1}.'.format(instrumento, var1))
randlook = xr.open_dataset(path_nc[random.randrange(1, len(path_nc)+1)])

Escolha o conjunto de dados que irá analisar. Para gases digite 1, aerossois 2, nuvens 3 e estado atmosferico 4: 4


### 2) Conferindo as dimensões e as variáveis de um netcdf (ou um xarray.Dataset) aleatório:

In [29]:
print('As dimensões são {0}.'.format(list(randlook.dims.keys())))
for d in range(len(list(randlook.dims.keys()))):
    if list(randlook.dims.keys())[d] == 'time':
        dim_time = list(randlook.dims.keys())[d]
        dim = dim_time
    else:
        if list(randlook.dims.keys())[d]:
            dim2 = list(randlook.dims.keys())[d]
            dim = dim2
        else:
            pass
randlook = xr.open_dataset(path_nc[random.randrange(1, len(path_nc)+1)])
randlook

As dimensões são ['time', 'size_bin', 'bound', 'n_lookup'].


<xarray.Dataset>
Dimensions:                            (time: 1440, size_bin: 20, bound: 2,
                                        n_lookup: 100)
Coordinates:
  * time                               (time) datetime64[ns] 2015-01-07 ... 2...
  * size_bin                           (size_bin) float32 0.0 1.0 ... 18.0 19.0
Dimensions without coordinates: bound, n_lookup
Data variables: (12/42)
    base_time                          datetime64[ns] ...
    time_offset                        (time) datetime64[ns] ...
    size_bin_bounds                    (size_bin, bound) float32 ...
    eta_lookup_table                   (n_lookup, bound) float32 ...
    seconds_after_transition           (time) float64 ...
    supersaturation_set_point          (time) float32 ...
    ...                                 ...
    N_CCN                              (time) float32 ...
    qc_N_CCN                           (time) int32 ...
    N_CCN_dN                           (time, size_bin) float32 ...
    lat                                float32 ...
    lon                                float32 ...
    alt                                float32 ...
Attributes: (12/20)
    command_line:             aosdilutioncorr -s mao -f M1 -b 20131211 -e 201...
    Conventions:              ARM-1.1
    process_version:          ingest-aosdilutioncorr-1.4-0.el6
    dod_version:              aosccn1col-b1-3.8
    site_id:                  mao
    facility_id:              M1
    ...                       ...
    Instrument:               CCN-100
    Instrument_Manufacturer:  Droplet Measurement Technologies
    AOS_DAC_type:             N
    serial_number:            N/A
    doi:                      10.5439/1227964
    history:                  created by user koontz on machine amber at 2015...

### 3) Processamento principal: Aqui são abertos todos os arquivos, aplicados os quality checks na variável de interesse, calculadas as médias (de minuto em minuto) e arrumados os buracos nos arquivos onde as medições não foram iniciadas às 00:00h. No final é gerada uma lista contendo listas, essas ultimas sendo os dias. Aqui não são verificados outliers!

In [30]:
import Dicionariosauxiliares as dcaux
import Retorna_xarrays2_smps_raw as retx

count = 0
colunas, ls_cru1,  = [], []
print('Há {0} arquivos na pasta {1}. Começando a processá-los:'.format(len(path_nc), path))

## Loop principal, aplicado nos arquivos:
for file in path_nc:
    nc = xr.open_dataset(file)
    if instrumento == 'arsclwacrrwp1fengM1':
        ano = file[-12:-8]
        if file[-6:-4][0] == '0':
            dia = file[-5:-4]
        else:
            dia = file[-6:-4]
        if file[-8:-6][0] == '0':
            mes = file[-7:-6]
        else:
            mes = file[-8:-6]
    else:
        ano = str(int(nc['time.year'].values[0]))
        dia = str(int(nc['time.day'].values[0]))
        mes = str(int(nc['time.month'].values[0]))
        hora = str(int(nc['time.hour'].values[0]))
        minuto = str(int(nc['time.minute'].values[0]))
        seg = str(int(nc['time.second'].values[0]))
    ## Lista de strings, cada string sendo o nome do dia:
    colunas.append(dia+'/'+mes+'/'+ano)
    ## Contando a quantidade de dias/arquivos lidos:
    count += 1
    ## Funções que criam os dicionarios os quais ajudarão a recortar dataframes de acordo com as datas:
    dic_aux_ano14 = dcaux.organiza_ano14(dia, mes, ano)
    dic_aux_ano15 = dcaux.organiza_ano15(dia, mes, ano)
    dic_aux_meses14 = dcaux.organiza_meses14(dia, mes, ano)
    dic_aux_meses15 = dcaux.organiza_meses15(dia, mes, ano)
    dic_aux_estacoes14 = dcaux.organiza_estacoes14(dia, mes, ano)
    dic_aux_estacoes15 = dcaux.organiza_estacoes15(dia, mes, ano)
    
    ## Função que retorna um xarray com a variável de interesse já com qc's aplicados:
    if instrumento in com_cortes:
        rawdata = retx.arrays(nc, var1, dim, corte)
    else:
        rawdata = retx.arrays(nc, var1, dim)
        
    ## Verificando a frequência na qual as medições foram feitas:
    if (len(rawdata.time) == 1440) or (len(rawdata.time) == 1):
        rawdata1 = rawdata
    elif len(rawdata.time) > 1440:
        ## Calculando médias de minuto em minuto:
        rawdata1 = rawdata.resample(time="1min").mean()
    elif (len(rawdata.time) < 1440) and (len(rawdata.time) > 1):
        ## Interpolando (linearmente) de minuto em minuto:
        rawdata1 = rawdata.resample(time='1min').interpolate('linear')
    elif (len(rawdata.time) == 0):
        oi = np.arange(0,1440,1,dtype=float)
        for i in range(len(oi)):
            oi[i] = np.nan
        rawdata1 = xr.DataArray(oi, dims=['time'])
        rawdata1.name = var1
        rawdata1.attrs = {'long_name': 'Dry Bulb Temperature','units': 'C','valid_min': -90.0,
                          'valid_max': 50.0,'valid_delta': 10.0,'resolution': 0.1}
        
    ## Pegando as médias calculadas e colocando em uma lista:
    list_rawdata1 = list(rawdata1.values)
    
    ## Setando o número total de minutos em um dia (60*24=1440):
    tot_p = 60*24
    
    ## Verificando se as medições se iniciaram à meia-noite. Em caso negativo, completando com NaN's:
    if instrumento == 'arsclwacrrwp1fengM1':
        nc.close()
        print(colored('{0}/{1}/{2}! -> {3}/{4} pontos.'.format(dia,mes,ano,len(list_rawdata1),tot_p),'blue'))
    else: 
        if hora != '0':
            print(colored('{0}/{1}/{2} - {3}:{4}:{5}! -> {6}/{7} pontos. Arrumando...'.format(dia,mes,ano,hora,
                                                                                          minuto,seg,
                                                                                          len(list_rawdata1),
                                                                                          tot_p),'red'))
        falta = tot_p - len(rawdata1.time)
        for pos in range(falta):
            list_rawdata1.insert(pos, np.nan)
        nc.close()
        print(colored('{0}/{1}/{2} - {3}:{4}:{5}! -> {6}/{7} pontos.'.format(dia,mes,ano,hora,minuto,seg,
                                                                         len(list_rawdata1),tot_p),'blue'))        
    ## Lista de listas, cada lista interna sendo um dia:
    ls_cru1.append(list_rawdata1)
    
## Printando o status do processamento na tela:
if instrumento == 'radflux1longM1':
    print(colored(u'ATENÇÃO! Para radflux TODAS se iniciam às 00:00 sim, o "04:00" está errado!', 'red'))
elif instrumento == 'sondewnpnM1':
    print(colored(u'ATENÇÃO! Para sonde TODAS se iniciam às 11:00, foi a sondagem que escolhi!', 'red'))
print(colored('{0}/{1} dias lidos. O tamanho de ls_cru1 é {0}.'.format(count,len(path_nc)),'green')) 

Há 458 arquivos na pasta /mnt/HD_500GB_WD/01_Dados_ARM/Aerossois/maoaosccn1colM1_b1. Começando a processá-los:
25/1/2014 - 22:52:56! -> 68/1440 pontos. Arrumando...
25/1/2014 - 22:52:56! -> 1440/1440 pontos.
26/1/2014 - 0:0:0! -> 1440/1440 pontos.
27/1/2014 - 0:0:0! -> 1440/1440 pontos.
28/1/2014 - 0:0:0! -> 1440/1440 pontos.
29/1/2014 - 0:0:0! -> 1440/1440 pontos.
30/1/2014 - 0:0:0! -> 1440/1440 pontos.
31/1/2014 - 0:0:0! -> 1440/1440 pontos.
1/2/2014 - 0:0:0! -> 1440/1440 pontos.
2/2/2014 - 0:0:0! -> 1440/1440 pontos.
3/2/2014 - 0:0:0! -> 1440/1440 pontos.
4/2/2014 - 0:0:0! -> 1440/1440 pontos.
5/2/2014 - 0:0:0! -> 1440/1440 pontos.
6/2/2014 - 0:0:0! -> 1440/1440 pontos.
7/2/2014 - 0:0:0! -> 1440/1440 pontos.
8/2/2014 - 0:0:0! -> 1440/1440 pontos.
9/2/2014 - 0:0:0! -> 1440/1440 pontos.
10/2/2014 - 0:0:0! -> 1440/1440 pontos.
11/2/2014 - 0:0:0! -> 1440/1440 pontos.
12/2/2014 - 0:0:0! -> 1440/1440 pontos.
13/2/2014 - 0:0:0! -> 1440/1440 pontos.
14/2/2014 - 0:0:0! -> 1440/1440 pontos.
1

9/7/2014 - 0:0:0! -> 1440/1440 pontos.
10/7/2014 - 0:0:0! -> 1440/1440 pontos.
11/7/2014 - 0:0:0! -> 1440/1440 pontos.
12/7/2014 - 0:0:0! -> 1440/1440 pontos.
13/7/2014 - 0:0:0! -> 1440/1440 pontos.
14/7/2014 - 0:0:0! -> 1440/1440 pontos.
15/7/2014 - 0:0:0! -> 1440/1440 pontos.
16/7/2014 - 0:0:0! -> 1440/1440 pontos.
17/7/2014 - 0:0:0! -> 1440/1440 pontos.
18/7/2014 - 0:0:0! -> 1440/1440 pontos.
19/7/2014 - 0:0:0! -> 1440/1440 pontos.
20/7/2014 - 0:0:0! -> 1440/1440 pontos.
21/7/2014 - 0:0:0! -> 1440/1440 pontos.
22/7/2014 - 0:0:0! -> 1440/1440 pontos.
23/7/2014 - 0:0:0! -> 1440/1440 pontos.
24/7/2014 - 0:0:0! -> 1440/1440 pontos.
25/7/2014 - 0:0:0! -> 1440/1440 pontos.
26/7/2014 - 0:0:0! -> 1440/1440 pontos.
27/7/2014 - 0:0:0! -> 1440/1440 pontos.
28/7/2014 - 0:0:0! -> 1440/1440 pontos.
29/7/2014 - 0:0:0! -> 1440/1440 pontos.
30/7/2014 - 0:0:0! -> 1440/1440 pontos.
31/7/2014 - 0:0:0! -> 1440/1440 pontos.
1/8/2014 - 0:0:0! -> 1440/1440 pontos.
2/8/2014 - 0:0:0! -> 1440/1440 pontos.
3/8

24/12/2014 - 0:0:0! -> 1440/1440 pontos.
25/12/2014 - 0:0:0! -> 1440/1440 pontos.
26/12/2014 - 0:0:0! -> 1440/1440 pontos.
27/12/2014 - 0:0:0! -> 1440/1440 pontos.
28/12/2014 - 0:0:0! -> 1440/1440 pontos.
29/12/2014 - 0:0:0! -> 1440/1440 pontos.
30/12/2014 - 0:0:0! -> 1440/1440 pontos.
31/12/2014 - 0:0:0! -> 1440/1440 pontos.
1/1/2015 - 0:0:0! -> 1440/1440 pontos.
2/1/2015 - 0:0:0! -> 1440/1440 pontos.
3/1/2015 - 0:0:0! -> 1440/1440 pontos.
4/1/2015 - 0:0:0! -> 1440/1440 pontos.
5/1/2015 - 0:0:0! -> 1440/1440 pontos.
6/1/2015 - 0:0:0! -> 1440/1440 pontos.
7/1/2015 - 0:0:0! -> 1440/1440 pontos.
8/1/2015 - 0:0:0! -> 1440/1440 pontos.
9/1/2015 - 0:0:0! -> 1440/1440 pontos.
10/1/2015 - 0:0:0! -> 1440/1440 pontos.
11/1/2015 - 0:0:0! -> 1440/1440 pontos.
12/1/2015 - 0:0:0! -> 1440/1440 pontos.
13/1/2015 - 0:0:0! -> 1440/1440 pontos.
14/1/2015 - 0:0:0! -> 1440/1440 pontos.
15/1/2015 - 0:0:0! -> 1440/1440 pontos.
16/1/2015 - 0:0:0! -> 1440/1440 pontos.
17/1/2015 - 0:0:0! -> 1440/1440 pontos.
1

### 4) Criando os dataframes com as médias de minuto em minuto já calculadas:

In [31]:
## Função que cria Pandas Dataframes 'normais' ou 'transpostos com datas nas colunas':
import Dataframesprincipais as dfpri
df_raw_1m = dfpri.dframe_normal_semdatas(ls_cru1)
df_T_dias_raw_1m = dfpri.dframe_transposto_comdatas(df_raw_1m, colunas)
df_T_dias_raw_1m

,25/1/2014,26/1/2014,27/1/2014,28/1/2014,29/1/2014,30/1/2014,31/1/2014,1/2/2014,2/2/2014,3/2/2014,...,5/5/2015,6/5/2015,7/5/2015,8/5/2015,9/5/2015,10/5/2015,11/5/2015,12/5/2015,13/5/2015,14/5/2015
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33.869999,NaN,NaN,NaN,NaN,178.989227,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,153.852127,47.500431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,NaN,NaN,NaN,NaN,307.363464,1797.090942,428.166870,NaN,NaN,NaN,...,118.766014,36.878963,92.458519,59.411480,53.132851,31.265844,152.783783,240.622574,267.591949,129.009995
1436,NaN,NaN,NaN,138.255203,301.334717,1788.093628,416.333862,NaN,NaN,NaN,...,125.356613,34.686333,94.743607,59.645496,58.462154,33.170597,149.480759,237.168427,268.730988,131.149994
1437,NaN,NaN,NaN,NaN,271.671356,1935.885010,386.613037,NaN,NaN,NaN,...,124.604706,35.969315,90.702248,61.008698,58.911041,37.357105,152.427811,238.464554,272.292542,132.190002
1438,NaN,NaN,NaN,NaN,274.092224,2013.527466,355.780182,NaN,NaN,NaN,...,123.743744,35.953098,89.353661,62.381248,59.235260,28.633255,157.147263,242.584030,271.123840,131.770004


### 5) Criando lista auxiliares e cheias: 

In [32]:
## Função que usa listas AUXILIARES para criar as listas CHEIAS a partir de um dataframe:
import Listascheias as lc

######################################### COM MÉDIAS 1 MIN. #################################################
## Listas AUXILIARES:
ls_aux_df_1m = [df_T_dias_raw_1m]

## Listas CHEIAS:
ls_cheia_1m = lc.lista_todos_valores_juntos(ls_aux_df_1m)

### 6) Criando dataframes CHEIOS (eles devem ser transpostos) para fazer boxplots:

In [33]:
## Dataframe com 1 coluna só contendo TODOS os dados: 
df_td_raw_1m = pd.DataFrame(data=ls_cheia_1m[0])

### 9) Exportando o dataframe com as médias de minuto em minuto:

In [34]:
lista_all_dfs = [df_T_dias_raw_1m]
lista_all_dfs_names = ['df_T_dias_raw_1m']

In [35]:
## Verificando o caminho onde será salvo odataframe com as médias:
directory0 = '/mnt/HD_500GB_WD/00_Doutorado/Codigos_Notebooks_Medias_Resample/Dataframes_Media1m/'
# directory0 = '/media/andre/HD_1000GB/00_Doutorado/Codigos_Notebooks_Medias_Resample/Dataframes_Media1m/'
if instrumento == 'sondewnpnM1':
    directory = directory0+instrumento+'_'+var1+'_'+lista_all_dfs_names[0]+'_1000hPa.pkl'
elif instrumento == 'arsclwacrbnd1kolliasM1':
    directory = directory0+instrumento[3:]+'_'+var1+'_layer_mais_baixa_detectada_'+lista_all_dfs_names[0]+'.pkl'
else:
    directory = directory0+instrumento[3:]+'_'+var1+'_'+lista_all_dfs_names[0]+'.pkl'
directory

'/mnt/HD_500GB_WD/00_Doutorado/Codigos_Notebooks_Medias_Resample/Dataframes_Media1m/ccn1colM1_b1_N_CCN_df_T_dias_raw_1m.pkl'

In [11]:
## Loop para gravar arquivos ".pkl" (tipo pikcle) a partir dos dataframes:
for i, df in enumerate(lista_all_dfs):
    if instrumento in com_cortes:
        pass
#         df.to_pickle(pasta_dfs+'Principal_Process/mao'+instrumento+'/'+instrumento[3:]+'_'+var1+'_'+corte+'_'+lista_all_dfs_names[i]+'.pkl')
#         df.to_pickle('/home/andre/Desktop/nefs'+'/'+instrumento[3:]+'_'+var1+'_'+corte+'_'+lista_all_dfs_names[i]+'.pkl')
#         df.to_pickle(pasta_dfs+instrumento[3:]+'_'+var1+'_'+corte+'_'+lista_all_dfs_names[i]+'.pkl')
    else:
        df.to_pickle(directory)